Задание:

Рассмотрим пример использования машинного обучения для домашней бухгалтерии. Применим машинное обучение для предсказания стоимости медицинской страховки для человека.

Для вас собрали и подготовили датасет с данными и шаблон в Colab-ноутбуке, который необходимо дополнить кодом.


Что нужно сделать:
1. перейдите по ссылке;
2. скопируйте ноутбук;
3. при нажатии «Скопировать на диск» Colab-ноутбук с шаблоном откроется в новом окне, работать необходимо в копии:
4. пункты задания находятся внутри ноутбука — они идут по порядку выполнения ячеек кода, комментарии начинаются с #;
5. результат выполненного задания — готовый ноутбук с предсказанием по конкретному человеку.


# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
df = pd.read_csv('insurance.csv', sep=',')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X, y = df[['age', 'sex', 'bmi', 'children', 'smoker',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest']],df['charges']

In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
1311,33,0,26.695,0,0,0,1,0,0
925,50,1,32.110,2,0,1,0,0,0
264,53,0,38.060,3,0,0,0,1,0
984,20,1,30.115,5,0,1,0,0,0
267,59,0,32.395,3,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
526,19,0,30.590,2,0,0,1,0,0
100,41,0,31.600,0,0,0,0,0,1
1120,23,0,31.400,0,1,0,0,0,1
402,64,0,32.965,0,0,0,1,0,0


In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
770,61,1,36.100,3,0,0,0,0,1
418,64,1,39.160,1,0,0,0,1,0
888,22,1,39.500,0,0,0,0,0,1
723,19,1,35.400,0,0,0,0,0,1
28,23,1,17.385,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
353,33,1,35.245,0,0,1,0,0,0
120,44,1,37.100,2,0,0,0,0,1
569,48,1,40.565,2,1,0,1,0,0
22,18,1,34.100,0,0,0,0,1,0


In [ ]:
y_train

1311     4571.41305
925     25333.33284
264     20462.99766
984      4915.05985
267     14590.63205
           ...     
526     24059.68019
100      6186.12700
1120    34166.27300
402     14692.66935
764      9095.06825
Name: charges, Length: 1070, dtype: float64

In [ ]:
y_test

770     27941.28758
418     14418.28040
888      1682.59700
723      1263.24900
28       2775.19215
           ...     
353     12404.87910
120      7740.33700
569     45702.02235
22       1137.01100
1222     8442.66700
Name: charges, Length: 268, dtype: float64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([  246.7103089 ,    54.26164516,   335.81720717,   390.37034121,
       24065.9223987 ,   593.42445314,   313.74419148,  -333.77745175,
        -573.39119286])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
lr.predict(X_test)

array([15856.08039247, 17082.68503179,  6205.04582599,  4088.06434989,
         302.66432392, 13764.53101166,   912.10528756, 39036.41513656,
       14381.16786352,  5919.92282914,  8170.04766409,  7216.9742917 ,
        8474.01297692,  4925.19725139, 32094.23607116,  6225.76762619,
       27319.85714424, 25600.00855131, 12679.76030269,  9467.2410686 ,
       15371.68134893, 38661.10431463, 10182.66611032,  5289.37883589,
        6416.09958917,  8999.30176698,  7500.68934528, 28961.28111673,
       18536.64318813, 14663.03865315, 11075.93343687, 35988.66235686,
       31236.0118802 , 15120.85129921, 25660.70133767, 23499.02936522,
       13148.62282494, 11176.05776026, 39965.76061971, 14576.71248323,
        6921.6502363 ,  3929.00874408, 17283.66366029,  5281.67658581,
       11183.69756322,  6648.27355   , 11587.84380504,  4715.66230365,
        8128.39773008,  4745.64131312, 31860.92651317,  8055.14991444,
        3801.46634014,  6566.48617562, 13077.40878323,  4588.85394385,
      

In [ ]:
pd.DataFrame([y_test.values, lr.predict(X_test)]).T

,0,1
0,27941.28758,15856.080392
1,14418.28040,17082.685032
2,1682.59700,6205.045826
3,1263.24900,4088.064350
4,2775.19215,302.664324
...,...,...
263,12404.87910,8656.772653
264,7740.33700,11607.452007
265,45702.02235,38710.957649
266,1137.01100,3644.405413


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки
mean_squared_error(y_test, lr.predict(X_test))

35960272.52413388

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

36735836.98055623

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([27607.63856232])